In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

Формат ссылки для пагинации https://lifehacker.ru/
Какой-то нумерации статьи на сайте не имеют, а сепарируются по транслитному названию в ссылке.

Ссылки на статьи хронятся в блоке с тегом "a" в классе "lh-small-article-card__link"

In [17]:
base_url = 'https://lifehacker.ru/topics/technology/'
parsed_urls = []
for page_num in range(1, 11):
    url = f'{base_url}/?page={page_num}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    raw_items = soup.find_all('a', class_='lh-small-article-card__link')
    parsed_urls += [item.get('href') for item in raw_items]

При отработке блока ниже в текст статьи попадают и блоки "Больше о наушниках", "Другие новинки" и подобные. В своём коде я отделила только "Новости из мира технологий". Можно ли как-то иначе прочищать текст от лишнего учаска, фактически не входящего в текст статьи? Весь этот блок входит в класс "read-also", но она, не знаю правильно ли так сказать, подкласс класса "single-article__post-content single-article__content-container"

In [37]:
base_url2 = 'https://lifehacker.ru'
result = []
for url in tqdm(parsed_urls):
    article = {}
    article_url = f'{base_url2}{url}' # подставляем часть ссылки, ведущую материал
    response = requests.get(article_url)
    soup = BeautifulSoup(response.text, 'lxml')
    article['title'] = soup.find('h1', class_='article-card__title').text
    art_text = soup.find('div', class_='single-article__post-content single-article__content-container').text
    article['text'] = art_text[:art_text.find('        Новости из мира технологий')]
    result.append(article)

100%|██████████| 300/300 [07:54<00:00,  1.58s/it]


[{'title': 'Названы лидеры на рынке носимых устройств.\xa0Apple и\xa0Xiaomi не удалось возглавить топ',
  'text': 'Маркетинговая компания IDC опубликовала отчёт о продажах носимых устройств в мире за первые три квартала 2024 года. Как оказалось, чаще всего пользователи выбирали умные часы и браслеты Huawei.\xa0Так, компания продала 23,6 миллиона единиц девайсов за указанный период, увеличив поставки на 7,3 миллиона по сравнению с прошлым годом. В итоге теперь она занимает 16,9% рынка.Аналитики полагают, что этот успех обусловлен недавним выпуском Watch GT5, Watch D2 и Watch Ultimate Green Edition, которые предлагают одни из лучших функций для мониторинга здоровья и обеспечивают высокую точность показателей.Apple заняла второе место, реализовав 22,5 миллиона устройств (16,2% рыночной доли). Продажи «яблочных» часов снизились на 12,8% по сравнению с аналогичным периодом прошлого года. Тройку лидеров замыкает Xiaomi с объёмом поставок в 20,5 миллиона единиц, долей рынка 14,7% и снижением 

In [39]:
pd.set_option('display.max_colwidth', 400)
data = pd.DataFrame(result)
data.head()

,title,text
0,Названы лидеры на рынке носимых устройств. Apple и Xiaomi не удалось возглавить топ,"Маркетинговая компания IDC опубликовала отчёт о продажах носимых устройств в мире за первые три квартала 2024 года. Как оказалось, чаще всего пользователи выбирали умные часы и браслеты Huawei. Так, компания продала 23,6 миллиона единиц девайсов за указанный период, увеличив поставки на 7,3 миллиона по сравнению с прошлым годом. В итоге теперь она занимает 16,9% рынка.Аналитики полагают, что э..."
1,Какой телевизор подойдёт для игр на PlayStation 5 и Xbox Series X/S,Оглавление \n В чём преимущества PlayStation 5 / 5 Pro и Xbox Series X/S\n \n На что смотреть при выборе телевизора для игр на PS5 / PS5 Pro и Xbox Series X/S\n \n Какой телевизор купить для PS5 / PS5 Pro и Xbox Series X/S\n Оглавление \n В чём преимущества PlayStation 5 / 5 Pro и Xbox Series X/S\n \n ...
2,"Представлен планшет Honor Pad V9 с 11,5-дюймовым экраном и 8 динамиками","Honor представила новый планшет на базе Android — Honor Pad V9. Толщина его металлического корпуса составляет 6,1 миллиметра, а вес — 475 граммов.Новинка получила 11,5-дюймовый ЖК-дисплей с разрешением 2800 x 1840 пикселей и частотой обновления 144 Гц. Он поддерживает ввод стилусом Magic Pencil 3, который продаётся отдельно.Изображение: HonorУстройство оснащено 13-мегапиксельной основной камер..."
3,Илон Маск анонсировал свой почтовый сервис Xmail с простым интерфейсом,"Илон Маск сообщил, что его команда работает над новым почтовым сервисом под названием Xmail, который будет отличаться упрощённым интерфейсом. Миллиардер хочет переосмыслить подход к обмену сообщениями. Нам нужно переосмыслить подход к обмену сообщениями, включая электронную почту. Эта задача уже в моём списке дел. Илон Маск По словам миллиардера, дизайн нового сервиса будет в стиле DM (Direct ..."
4,"Не надо так: 7 привычек, которые убивают бытовую технику","1. Загружать стиральную машину до упораСамый простой способ сократить срок службы стиралки — загружать её до максимума, игнорируя рекомендации производителя. Желание совершенно понятное, особенно когда за день нужно перестирать гораздо больше вещей, чем можно загрузить за один раз (но впихнуть всё же реально).Только вот хорошо от этого не будет никому. Одежде нужно свободное место, чтобы перем..."
